In [ ]:
#################################################################################
##########LDA##########
#################################################################################

### import kaggle description data

library(readr)
kaggdescr <- read_csv("~/Workingdirectory/Data/kaggdescr.csv", 
                      col_types = cols(X1 = col_skip()))
View(kaggdescr)

library(tidytext)
library(dplyr)
library(gutenbergr)

compid <- kaggdescr['Competition Id']
desc <- kaggdescr['Overview']

###################################

library('text2vec')

library(topicmodels)
chapters_lda <- LDA(chapters_dtm, k = 4, control = list(seed = 1234))
chapters_lda

create_dtm(kaggdescr)


### calculate dtm

setDT(kaggdescr)
setkey(kaggdescr,sic)
train=kaggdescr 
test=kaggdescr
View(train)

prep_fun=tolower
tok_fun=word_tokenizer


it_train= itoken(train$Overview,
                 preprocessor = prep_fun,
                 tokenizer= tok_fun,
                 ids = train$`Competition Id`)

vocab=create_vocabulary(it_train)
#dim(vocab)
vectorizer=vocab_vectorizer(vocab)
dtm_train=create_dtm(it_train,vectorizer)

it_test = itoken(test$Overview,
                 preprocessor=prep_fun,
                 tokenizer=tok_fun,
                 ids=test$`Competition Id`)

dtm_test = create_dtm(it_test, vectorizer )


####################################

by_chapter <- desc %>%
  group_by(compid) %>%
  mutate(chapter = )

############


library(tidytext)
library(stringr)
library(tidyr)

by_chapter_word <- kaggdescr %>%
  unite(compid,desc) %>%
  unnest_tokens(word,text)

unnest_tokens(books)

############################

library(tm)
data("AssociatedPress", package = "topicmodels")
AssociatedPress
ap_td <-tidy(AssociatedPress)

############################

library(tidytext)
library(stringr)
library(tidyr)

by_chapter <- kaggdescr %>%
  group_by(kaggdescr$`Competition Id`) %>%
  mutate(overview = cumsum(kaggdescr$Overview)) %>%
  ungroup() 

kaggle_td <- tidy(kaggdescr) 

#######################################################################
#################### data processing for LDA ##########################
#######################################################################

library(tm)
kaggdescr<- kaggdescr[complete.cases(kaggdescr),]

#kaggle_proc <- tm_map(kaggdescr,content_transformer(tolower))

vs = VectorSource(kaggdescr$Overview)
kagg_corpus <- Corpus(vs,readerControl = list(readPlain, language = 'en', load = TRUE))

### make text lowercase
kagg_corpus_proc <- tm_map(kagg_corpus,content_transformer(tolower))
### remove stopwords
kagg_corpus_proc <- tm_map(kagg_corpus_proc, removeWords, stopwords( kind = 'en'))
### remove punctuations
kagg_corpus_proc <- tm_map(kagg_corpus_proc, removePunctuation)
### remove numbers
kagg_corpus_proc <- tm_map(kagg_corpus_proc, removeNumbers)
### strip whitespace
kagg_corpus_proc <- tm_map(kagg_corpus_proc, stripWhitespace)
### create dtm
dtm <- DocumentTermMatrix(kagg_corpus_proc)

####### create bigrams#######
library(RWeka)
library(rJava)

options(mc.cores = 1)
twogramTokenizer <- function(x) {
  NGramTokenizer(x,Weka_control(min = 2, max = 3))
}

################################################################################
################################## LDA #########################################
################################################################################

library(topicmodels)
lda_kagg <- LDA(dtm, k = 10, control = list(seed = 1234, alpha = 0.1),tokenize = twogramTokenizer())

################################################################################
kaggle_lda_td <- tidy(kagg_lda)
kaggle_lda_td

#topics(kagg_lda, 4, 0.5)
####### get top 5 terms for each topic
top_terms <- kaggle_lda_td %>%
  group_by(topic) %>%
  top_n(10,beta) %>%
  ungroup() %>%
  arrange(topic, -beta)

top_terms

library(ggplot2)
theme_set(theme_bw())

top_terms %>%
  mutate(term = reorder(term,beta)) %>%
  ggplot(aes(term,beta)) +
  geom_bar(stat = 'identity') +
  facet_wrap(~topic, scales = 'free') +
  theme(axis.text.x = element_text(size = 15, angle = 90, hjust = 1))


###########

kaggle_lda_gamma <- tidy(lda_kagg, matrix = 'gamma' )
kaggle_lda_gamma

#### plot gammas
ggplot(kaggle_lda_gamma, aes(gamma, fill = factor(topic))) +
  geom_histogram() +
  facet_wrap(~ document, nrow = 2)

#### score each document on topics

kaggle_classifications <- kaggle_lda_gamma %>%
  group_by(document) %>%
  top_n(5,gamma) %>%
  arrange(gamma)

kaggle_classifications


########################### ## score part 2## #############################

gammaDF <- as.data.frame(lda_kagg@gamma) 
#names(gammaDF) <- c(1:10)

kaggletopics <- cbind(kaggdescr, gammaDF )

setnames(kaggletopics, old = c("V1","V2","V3","V4","V5","V6","V7","V8","V9","V10"), 
         new =c("topic1","topic2", "topic3","topic4", "topic5", "topic6","topic7","topic8","topic9","topic10"))


#### merge with original doc ##


